In [ ]:
# Visualize data using tensorboard's embedding projector
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

%load_ext tensorboard

In [ ]:
import os
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorboard.plugins import projector
import pandas as pd

In [ ]:
# load data
df = pd.read_csv('data/cluster_df.csv')

In [ ]:
X = df.iloc[:, :-1] # take all but the last feature of df and store in X

In [ ]:
# set up a logs directory
log_dir = './logs/vis-example'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# create function to save labels
def register_embedding(embedding_tensor_name, meta_data_fname, log_dir):
    config = projector.ProjectorConfig()
    embedding = config.embeddings.add()
    embedding.tensor_name = embedding_tensor_name
    embedding.metadata_path = meta_data_fname
    projector.visualize_embeddings(log_dir, config)

def save_labels_tsv(labels, filepath, log_dir):
    with open(os.path.join(log_dir, filepath), 'w') as f:
        for label in labels:
            f.write('{}\n'.format(label))

In [ ]:
C_vals = df.iloc[:, -1] # store cluster assignments

META_DATA_FNAME = 'meta_cos_reduction1.tsv'  # Labels will be stored here
EMBEDDINGS_TENSOR_NAME = 'sample_data_cos_reduction1'
EMBEDDINGS_FPATH = os.path.join(log_dir, EMBEDDINGS_TENSOR_NAME + '.ckpt')
STEP = 0

register_embedding(EMBEDDINGS_TENSOR_NAME, META_DATA_FNAME, log_dir)
save_labels_tsv(C_vals, META_DATA_FNAME, log_dir)#version 1

In [ ]:
tf.compat.v1.disable_eager_execution()
tensor_embeddings = tf.Variable(X, name=EMBEDDINGS_TENSOR_NAME)
sess = tf.compat.v1.InteractiveSession()
sess.run(tf.compat.v1.global_variables_initializer())
saver = tf.compat.v1.train.Saver()
saver.save(sess, EMBEDDINGS_FPATH, STEP)
sess.close()

In [ ]:
# Clear any logs from previous runs
!remove -rf ./logs/

In [15]:
%tensorboard --logdir /logs/vis-example/

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 5632.